In [7]:
import xml.etree.ElementTree as ET
import json

tree = ET.parse('informatica_komahar.XML')
root = tree.getroot()

project_name = "PROJECT 1"

In [8]:

sessions = []
mappings = []

print("Sessions are:")
for session in root.iter('SESSION'):
    sessions.append(session.attrib)
    print(session.attrib)

print("\nMappings are:")
for mapping in root.iter('MAPPING'):
    mappings.append(mapping.attrib)
    print(mapping.attrib)


Sessions are:
{'DESCRIPTION': '', 'ISVALID': 'YES', 'MAPPINGNAME': 'm_Do_PostBatch_Workflow', 'NAME': 's_m_Do_PostBatch_Workflow_AgencyHierarchyDM', 'REUSABLE': 'YES', 'SORTORDER': 'Binary', 'VERSIONNUMBER': '6'}
{'DESCRIPTION': 'This session is used to load the Missing Agency Standalone Records', 'ISVALID': 'YES', 'MAPPINGNAME': 'm_AGY_DM_LOAD_AgencyRelationshipCurrent', 'NAME': 's_m_AGY_DM_LOAD_AgencyRelationshipCurrent', 'REUSABLE': 'YES', 'SORTORDER': 'Binary', 'VERSIONNUMBER': '4'}
{'DESCRIPTION': 'This session used to load data into agencyrelationshipdim table sourcing from rpt_edm..agencyrelationship table.this session maintains type 2 logic created in the mapping.', 'ISVALID': 'YES', 'MAPPINGNAME': 'm_AGY_DM_LOAD_AgencyRelationshipDim', 'NAME': 's_m_AGY_DM_LOAD_AgencyRelationshipDim', 'REUSABLE': 'YES', 'SORTORDER': 'Binary', 'VERSIONNUMBER': '2'}
{'DESCRIPTION': '', 'ISVALID': 'YES', 'MAPPINGNAME': 'm_AGY_DM_LOAD_AgencySilverCircleTierDim', 'NAME': 's_m_AGY_DM_LOAD_AgencySilve

In [9]:
print("Session example:\n", sessions[2])

Session example:
 {'DESCRIPTION': 'This session used to load data into agencyrelationshipdim table sourcing from rpt_edm..agencyrelationship table.this session maintains type 2 logic created in the mapping.', 'ISVALID': 'YES', 'MAPPINGNAME': 'm_AGY_DM_LOAD_AgencyRelationshipDim', 'NAME': 's_m_AGY_DM_LOAD_AgencyRelationshipDim', 'REUSABLE': 'YES', 'SORTORDER': 'Binary', 'VERSIONNUMBER': '2'}


### Code to find required <MAPPING> tag by checking its NAME equals "m_AGY_DM...."  and store it including its all children tags.

In [10]:
# Find the required <MAPPING> tag
required_mapping = None
for mapping in root.findall('.//MAPPING'):
    if mapping.get('NAME') == 'm_AGY_DM_Load_AgencyDim_v3':
        required_mapping = mapping
        break

# Store the required <MAPPING> tag including its children
if required_mapping is not None:
    mapping_as_string = ET.tostring(required_mapping, encoding='unicode')
    print(mapping_as_string)
else:
    print("The required <MAPPING> tag was not found.")

<MAPPING DESCRIPTION="The main source for the mapping is V2.Agency table and the target is WC_Data_Mart.V3.AgencyDim.&#13;&#10;This is basically type 2 Logic mapping where we are maintaining complete history on a given agency level. The HashKey columns in the V3.AgencyDim table are AgencyCode, AgencyLegalName, AgencyStatusCode, AgencyAppointedDate, AgencyTerminatedDate, PrimaryAgencyIndicator &amp; PhysicalZipCode.&#13;&#10;Filters on source:&#13;&#10;We are reading those agency data from V2.Agency whose where CurrentSnapshotFlag = 1" ISVALID="YES" NAME="m_AGY_DM_Load_AgencyDim_v3" OBJECTVERSION="1" VERSIONNUMBER="7">
        <TRANSFORMATION DESCRIPTION="Lookup Table dbo.AgencyAddress which CurrentSnapshotFlag is 1 and AddressType is 'Physical'" NAME="LKP_PhysicalAddress" OBJECTVERSION="1" REUSABLE="NO" TYPE="Lookup Procedure" VERSIONNUMBER="6">
            <TRANSFORMFIELD DATATYPE="integer" DEFAULTVALUE="" DESCRIPTION="" NAME="in_AgencyAKID" PICTURETEXT="" PORTTYPE="INPUT" PRECISION="

## XML to Model(JSON conversion)

In [11]:
model = {}

sessions = []
mappings = []

for session in root.iter('SESSION'):
    sessions.append(session.attrib)
for mapping in root.iter('MAPPING'):
    mappings.append(mapping.attrib)

# editing MAPPINGNAME of some sessions because they contain `_workflow` in their MAPPINGNAME 
# so the actual way is to search for <SHORTCUT> tag with that MappingName and then finally to <MAPPING>. But here I've used a simple hack here by erasing the `_Workflow` from the MAppingName
mappingName_from_session = { session["NAME"]: session['MAPPINGNAME'].replace('_Workflow', '') for session in sessions}
# print(mappingName_from_session)


for i, workflow in enumerate(root.iter('WORKFLOW')):
    # workflow_id = i
    workflow_name = workflow.get('NAME')
    workflow_description = workflow.get('DESCRIPTION')
    tasks = {}

    for j, task in enumerate(workflow.iter('TASKINSTANCE')):
        # task_id = j
        task_type = task.get('TASKTYPE')
        name = task.get('NAME')
        task_name = task.get('TASKNAME')
        execution_order = 0

        if task_type == "Start":
            # tasks[task_id] = {'task_id': task_id, "task_type": task_type, "name": name, "task_name": task_name, "execution_order": execution_order, "mapping": {}}
            tasks[name] = {"task_type": task_type, "name": name, "task_name": task_name, "execution_order": execution_order, "mapping": {}}
        else:
            mapping = {}

            required_mapping = None
            for temp_mapping in root.findall('.//MAPPING'):
                if temp_mapping.get('NAME') == mappingName_from_session[task_name]:
                    required_mapping = temp_mapping
                    break
            mapping_name = required_mapping.get('NAME')
            mapping_description = required_mapping.get('DESCRIPTION')

            instances = {}
            port_values = {
                "INPUT": {"is_input": 1, "is_output": 0, "is_lookup": 0},
                "LOOKUP": {"is_input": 0, "is_output": 0, "is_lookup": 1},
                "OUTPUT": {"is_input": 0, "is_output": 1, "is_lookup": 0},
                "INPUT/OUTPUT": {"is_input": 1, "is_output": 1, "is_lookup": 0},
                "LOOKUP/OUTPUT": {"is_input": 0, "is_output": 1, "is_lookup": 1},
                "LOOKUP/RETURN/OUTPUT": {"is_input": 0, "is_output": 1, "is_lookup": 1}
            }
            transformations_field = {trans.get('NAME'): [{'field_name': field.get('NAME'), 'field_type': field.get('DATATYPE'), 'port_values': port_values.get(field.get('PORTTYPE'), {"is_input": 0, "is_output": 0, "is_lookup": 0})} for field in trans.iter('TRANSFORMFIELD')] for trans in required_mapping.iter('TRANSFORMATION')}
            targets_field = {target.get('NAME'): [{'field_name': field.get('NAME'), 'field_type': field.get('DATATYPE'), 'port_values': {"is_input": 0, "is_output": 1, "is_lookup": 0}} for field in target.iter('TARGETFIELD')] for target in root.iter('TARGET')}
            sources_field = {source.get('NAME'): [{'field_name': field.get('NAME'), 'field_type': field.get('DATATYPE'), 'port_values': {"is_input": 1, "is_output": 0, "is_lookup": 0}} for field in source.iter('TARGETFIELD')] for source in root.iter('SOURCE')}
        
            for instance in required_mapping.iter('INSTANCE'):
                instance_name = instance.get('NAME')
                transformation_type = instance.get('TRANSFORMATION_TYPE')
                transformation_name = instance.get('TRANSFORMATION_NAME')
                # getting `TYPE` attribute to know where to look for field attributes
                type = instance.get('TYPE')
                # execution_order =0
                # fields = transformations_field.get(transformation_name, [])
                if type == "TRANSFORMATION":
                    fields = transformations_field[transformation_name]
                elif type == "TARGET":
                    for temp_target in root.findall('.//TARGET'):
                        if temp_target.get('NAME') == transformation_name:
                            fields = targets_field[transformation_name]
                            break
                elif type == "SOURCE":
                    for temp_source in root.findall('.//SOURCE'):
                        if temp_source.get('NAME') == transformation_name:
                            fields = sources_field[transformation_name]
                            break
                else:
                    fields = []
                instances[instance_name] = {'instance_name': instance_name, 'transformation_type': transformation_type, 'transformation_name': transformation_name, 'execution_order': 0, 'fields': fields}
            tasks[name] = {"task_name": task_name, "task_type": task_type, "name": name, "execution_order": execution_order, "mapping": {"mapping_name": mapping_name, "mapping_description": mapping_description, "instances": instances}}

            
        model[workflow_name] = {'workflow_name': workflow_name, 'project': project_name, 'workflow_description': workflow_description, 'tasks': tasks}
            


## Saving JSON

In [12]:
# Serializing json
json_object = json.dumps(model, indent=4)
 
# Writing to sample.json
with open("model_output.json", "w") as outfile:
    outfile.write(json_object)